# Feature Engineering
Cleaning features and predicting them before going to the AHEI forecast.
We will be using Prophet to fill the gaps between years in the features. Prophet was chosen
beause of explanability, backwards forecast, and seasonality fitting.

In [1]:
import pandas as pd
from prophet import Prophet
import numpy as np
import multiprocessing

C:\Users\Nickolas Machado\miniconda3\envs\onu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_parquet("data/processed/features.parquet")

In [3]:
# Removing everything before 2000 because its too old and most datasets don't have
# data past it
df = df[df["Year"] > 2000]
df

,Country,Year,AHEI,HCI,CPI,CPIFoodEnergy,HealthyDietCost,FruitsShare,VegetablesShare,StarchyShare,AnimalSourceShare,LegumesShare,OilsShare
11,AFG,2001,48.266802,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,AFG,2002,47.970130,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13,AFG,2003,47.673459,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14,AFG,2004,47.376788,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15,AFG,2005,47.080117,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,WLD,2020,NaN,<NA>,<NA>,<NA>,3.537,0.131875,0.2005,0.25525,0.299375,0.072625,0.041375
7167,ZMB,2019,NaN,<NA>,<NA>,<NA>,3.245,0.1345,0.2425,0.2115,0.2845,0.085,0.042
7168,ZMB,2020,NaN,<NA>,<NA>,<NA>,3.3,0.14125,0.26125,0.18525,0.27975,0.09,0.042
7169,ZWE,2019,NaN,<NA>,<NA>,<NA>,3.456,0.148,0.28,0.159,0.275,0.095,0.042


## Predicting food prices

In [5]:
years = pd.DataFrame({"ds": range(2000, 2025+1)})

In [6]:
features = pd.DataFrame()
for country, group_df in df.groupby("Country"):
    fit = True
    country_df = pd.DataFrame()
    for col in df.columns:
        if col not in ["AHEI", "Country", "Year"]:
            m = Prophet()
            df_pred = group_df[["Year", col]].rename(columns={"Year": "ds", col: "y"})
            if not all(df_pred["y"].isna()):
                df_pred = df_pred.merge(years, on="ds", how="right")
                try:
                    m.fit(df_pred)
                    country_df["Country"] = country
                    country_df["Year"] = years["ds"]
                    country_df[col] = m.predict(df_pred)["yhat"]
                except:
                    fit = False
                    print(f"{country} - errored")
                    break
    if fit:
        print(f"{country} - processed")
        features = pd.concat([features, country_df])
features

17:23:09 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:13 - cmdstanpy - INFO - Chain [1] start processing
17:23:13 - cmdstanpy - INFO - Chain [1] done processing
17:23:13 - cmdstanpy - INFO - Chain [1] start processing
17:23:13 - cmdstanpy - INFO - Chain [1] done processing
17:23:14 - cmdstanpy - INFO - Chain [1] start processing
17:23:14 - cmdstanpy - INFO - Chain [1] done processing
17:23:15 - cmdstanpy - INFO - Chain [1] start processing
17:23:15 - cmdstanpy - INFO - Chain [1] done processing
17:23:15 - cmdstanpy - INFO - Chain [1] start processing
17:23:15 - cmdstanpy - INFO - Chain [1] done processing
17:23:15 - cmdstanpy - INFO - Chain [1] start processing


ABW - processed


17:23:17 - cmdstanpy - INFO - Chain [1] done processing
17:23:18 - cmdstanpy - INFO - Chain [1] start processing


AFG - processed


17:23:21 - cmdstanpy - INFO - Chain [1] done processing
17:23:21 - cmdstanpy - INFO - Chain [1] start processing
17:23:23 - cmdstanpy - INFO - Chain [1] done processing
17:23:24 - cmdstanpy - INFO - Chain [1] start processing
17:23:24 - cmdstanpy - INFO - Chain [1] done processing
17:23:24 - cmdstanpy - INFO - Chain [1] start processing
17:23:24 - cmdstanpy - INFO - Chain [1] done processing
17:23:25 - cmdstanpy - INFO - Chain [1] start processing
17:23:25 - cmdstanpy - INFO - Chain [1] done processing
17:23:25 - cmdstanpy - INFO - Chain [1] start processing
17:23:26 - cmdstanpy - INFO - Chain [1] done processing
17:23:26 - cmdstanpy - INFO - Chain [1] start processing
17:23:26 - cmdstanpy - INFO - Chain [1] done processing
17:23:27 - cmdstanpy - INFO - Chain [1] start processing
17:23:27 - cmdstanpy - INFO - Chain [1] done processing


...

17:45:57 - cmdstanpy - INFO - Chain [1] start processing
17:45:57 - cmdstanpy - INFO - Chain [1] done processing
17:45:58 - cmdstanpy - INFO - Chain [1] start processing
17:45:58 - cmdstanpy - INFO - Chain [1] done processing
17:45:58 - cmdstanpy - INFO - Chain [1] start processing
17:46:02 - cmdstanpy - INFO - Chain [1] done processing
17:46:02 - cmdstanpy - INFO - Chain [1] start processing
17:46:02 - cmdstanpy - INFO - Chain [1] done processing
17:46:03 - cmdstanpy - INFO - Chain [1] start processing
17:46:03 - cmdstanpy - INFO - Chain [1] done processing
17:46:03 - cmdstanpy - INFO - Chain [1] start processing
17:46:03 - cmdstanpy - INFO - Chain [1] done processing
17:46:04 - cmdstanpy - INFO - Chain [1] start processing
17:46:04 - cmdstanpy - INFO - Chain [1] done processing
17:46:04 - cmdstanpy - INFO - Chain [1] start processing
17:46:04 - cmdstanpy - INFO - Chain [1] done processing
17:46:05 - cmdstanpy - INFO - Chain [1] start processing
17:46:05 - cmdstanpy - INFO - Chain [1]

ZAF - processed


17:46:07 - cmdstanpy - INFO - Chain [1] done processing
17:46:08 - cmdstanpy - INFO - Chain [1] start processing
17:46:11 - cmdstanpy - INFO - Chain [1] done processing
17:46:11 - cmdstanpy - INFO - Chain [1] start processing
17:46:11 - cmdstanpy - INFO - Chain [1] done processing
17:46:12 - cmdstanpy - INFO - Chain [1] start processing
17:46:12 - cmdstanpy - INFO - Chain [1] done processing
17:46:12 - cmdstanpy - INFO - Chain [1] start processing
17:46:12 - cmdstanpy - INFO - Chain [1] done processing
17:46:13 - cmdstanpy - INFO - Chain [1] start processing
17:46:13 - cmdstanpy - INFO - Chain [1] done processing
17:46:13 - cmdstanpy - INFO - Chain [1] start processing
17:46:13 - cmdstanpy - INFO - Chain [1] done processing
17:46:14 - cmdstanpy - INFO - Chain [1] start processing
17:46:14 - cmdstanpy - INFO - Chain [1] done processing


ZMB - processed
ZWE - processed


,Country,Year,HealthyDietCost,FruitsShare,VegetablesShare,StarchyShare,AnimalSourceShare,LegumesShare,OilsShare,HCI,CPI,CPIFoodEnergy
0,ABW,2000,-0.016010,-0.524,0.26925,0.80675,0.23975,0.15725,0.051,NaN,NaN,NaN
1,ABW,2001,0.185991,-0.485,0.26900,0.77300,0.23800,0.15500,0.050,NaN,NaN,NaN
2,ABW,2002,0.387991,-0.446,0.26875,0.73925,0.23625,0.15275,0.049,NaN,NaN,NaN
3,ABW,2003,0.589992,-0.407,0.26850,0.70550,0.23450,0.15050,0.048,NaN,NaN,NaN
4,ABW,2004,0.791993,-0.368,0.26825,0.67175,0.23275,0.14825,0.047,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
21,ZWE,2021,3.456000,0.148,0.28000,0.15900,0.27500,0.09500,0.042,0.471359,NaN,NaN
22,ZWE,2022,3.456000,0.148,0.28000,0.15900,0.27500,0.09500,0.042,0.476989,NaN,NaN
23,ZWE,2023,3.456000,0.148,0.28000,0.15900,0.27500,0.09500,0.042,0.482620,NaN,NaN
24,ZWE,2024,3.456000,0.148,0.28000,0.15900,0.27500,0.09500,0.042,0.488250,NaN,NaN


In [19]:
features = features[~features["HCI"].isna() & ~features["CPI"].isna()].drop(columns=["CPIFoodEnergy"])
features

,Country,Year,HealthyDietCost,FruitsShare,VegetablesShare,StarchyShare,AnimalSourceShare,LegumesShare,OilsShare,HCI,CPI
0,ARG,2000,4.382250,0.47825,0.507500,-0.005500,-0.284501,0.28525,0.013750,0.556496,-15.696295
1,ARG,2001,4.321000,0.46300,0.490000,0.003000,-0.256001,0.28000,0.015000,0.559702,-12.606379
2,ARG,2002,4.259750,0.44775,0.472500,0.011500,-0.227501,0.27475,0.016250,0.562908,-9.516463
3,ARG,2003,4.198500,0.43250,0.455000,0.020000,-0.199001,0.26950,0.017500,0.566114,-6.426547
4,ARG,2004,4.137250,0.41725,0.437500,0.028500,-0.170501,0.26425,0.018750,0.569321,-3.336631
...,...,...,...,...,...,...,...,...,...,...,...
21,ZAF,2021,4.397999,0.23600,0.222500,0.128500,0.244500,0.14300,0.025500,0.410539,4.793088
22,ZAF,2022,4.497999,0.25075,0.235375,0.122125,0.241125,0.12675,0.024125,0.409302,4.756091
23,ZAF,2023,4.597999,0.26550,0.248250,0.115750,0.237750,0.11050,0.022750,0.408065,4.719093
24,ZAF,2024,4.697999,0.28025,0.261125,0.109375,0.234375,0.09425,0.021375,0.406828,4.682095


In [22]:
df_final = features.merge(df[["Country", "Year", "AHEI"]], how="left")
df_final = df_final.query("Year > 2000")
df_final

,Country,Year,HealthyDietCost,FruitsShare,VegetablesShare,StarchyShare,AnimalSourceShare,LegumesShare,OilsShare,HCI,CPI,AHEI
1,ARG,2001,4.321000,0.46300,0.490000,0.003000,-0.256001,0.28000,0.015000,0.559702,-12.606379,33.192833
2,ARG,2002,4.259750,0.44775,0.472500,0.011500,-0.227501,0.27475,0.016250,0.562908,-9.516463,33.061453
3,ARG,2003,4.198500,0.43250,0.455000,0.020000,-0.199001,0.26950,0.017500,0.566114,-6.426547,32.930073
4,ARG,2004,4.137250,0.41725,0.437500,0.028500,-0.170501,0.26425,0.018750,0.569321,-3.336631,32.798693
5,ARG,2005,4.076000,0.40200,0.420000,0.037000,-0.142001,0.25900,0.020000,0.572527,-0.246715,32.667312
...,...,...,...,...,...,...,...,...,...,...,...,...
1191,ZAF,2021,4.397999,0.23600,0.222500,0.128500,0.244500,0.14300,0.025500,0.410539,4.793088,NaN
1192,ZAF,2022,4.497999,0.25075,0.235375,0.122125,0.241125,0.12675,0.024125,0.409302,4.756091,NaN
1193,ZAF,2023,4.597999,0.26550,0.248250,0.115750,0.237750,0.11050,0.022750,0.408065,4.719093,NaN
1194,ZAF,2024,4.697999,0.28025,0.261125,0.109375,0.234375,0.09425,0.021375,0.406828,4.682095,NaN


In [23]:
df_final.to_parquet("data/model_input/features.parquet")